In [83]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [10]:
%sendtofile ssd1306.py

# MicroPython SSD1306 OLED driver, I2C and SPI interfaces

from micropython import const
import framebuf


# register definitions
SET_CONTRAST        = const(0x81)
SET_ENTIRE_ON       = const(0xa4)
SET_NORM_INV        = const(0xa6)
SET_DISP            = const(0xae)
SET_MEM_ADDR        = const(0x20)
SET_COL_ADDR        = const(0x21)
SET_PAGE_ADDR       = const(0x22)
SET_DISP_START_LINE = const(0x40)
SET_SEG_REMAP       = const(0xa0)
SET_MUX_RATIO       = const(0xa8)
SET_COM_OUT_DIR     = const(0xc0)
SET_DISP_OFFSET     = const(0xd3)
SET_COM_PIN_CFG     = const(0xda)
SET_DISP_CLK_DIV    = const(0xd5)
SET_PRECHARGE       = const(0xd9)
SET_VCOM_DESEL      = const(0xdb)
SET_CHARGE_PUMP     = const(0x8d)

# Subclassing FrameBuffer provides support for graphics primitives
# http://docs.micropython.org/en/latest/pyboard/library/framebuf.html
class SSD1306(framebuf.FrameBuffer):
    def __init__(self, width, height, external_vcc):
        self.width = width
        self.height = height
        self.external_vcc = external_vcc
        self.pages = self.height // 8
        self.buffer = bytearray(self.pages * self.width)
        super().__init__(self.buffer, self.width, self.height, framebuf.MONO_VLSB)
        self.init_display()

    def init_display(self):
        for cmd in (
            SET_DISP | 0x00, # off
            # address setting
            SET_MEM_ADDR, 0x00, # horizontal
            # resolution and layout
            SET_DISP_START_LINE | 0x00,
            SET_SEG_REMAP | 0x01, # column addr 127 mapped to SEG0
            SET_MUX_RATIO, self.height - 1,
            SET_COM_OUT_DIR | 0x08, # scan from COM[N] to COM0
            SET_DISP_OFFSET, 0x00,
            SET_COM_PIN_CFG, 0x02 if self.height == 32 else 0x12,
            # timing and driving scheme
            SET_DISP_CLK_DIV, 0x80,
            SET_PRECHARGE, 0x22 if self.external_vcc else 0xf1,
            SET_VCOM_DESEL, 0x30, # 0.83*Vcc
            # display
            SET_CONTRAST, 0xff, # maximum
            SET_ENTIRE_ON, # output follows RAM contents
            SET_NORM_INV, # not inverted
            # charge pump
            SET_CHARGE_PUMP, 0x10 if self.external_vcc else 0x14,
            SET_DISP | 0x01): # on
            self.write_cmd(cmd)
        self.fill(0)
        self.show()

    def poweroff(self):
        self.write_cmd(SET_DISP | 0x00)

    def poweron(self):
        self.write_cmd(SET_DISP | 0x01)

    def contrast(self, contrast):
        self.write_cmd(SET_CONTRAST)
        self.write_cmd(contrast)

    def invert(self, invert):
        self.write_cmd(SET_NORM_INV | (invert & 1))

    def show(self):
        x0 = 0
        x1 = self.width - 1
        if self.width == 64:
            # displays with width of 64 pixels are shifted by 32
            x0 += 32
            x1 += 32
        self.write_cmd(SET_COL_ADDR)
        self.write_cmd(x0)
        self.write_cmd(x1)
        self.write_cmd(SET_PAGE_ADDR)
        self.write_cmd(0)
        self.write_cmd(self.pages - 1)
        self.write_data(self.buffer)


class SSD1306_I2C(SSD1306):
    def __init__(self, width, height, i2c, addr=0x3c, external_vcc=False):
        self.i2c = i2c
        self.addr = addr
        self.temp = bytearray(2)
        super().__init__(width, height, external_vcc)

    def write_cmd(self, cmd):
        self.temp[0] = 0x80 # Co=1, D/C#=0
        self.temp[1] = cmd
        self.i2c.writeto(self.addr, self.temp)

    def write_data(self, buf):
        self.temp[0] = self.addr << 1
        self.temp[1] = 0x40 # Co=0, D/C#=1
        self.i2c.start()
        self.i2c.write(self.temp)
        self.i2c.write(buf)
        self.i2c.stop()


class SSD1306_SPI(SSD1306):
    def __init__(self, width, height, spi, dc, res, cs, external_vcc=False):
        self.rate = 10 * 1024 * 1024
        dc.init(dc.OUT, value=0)
        res.init(res.OUT, value=0)
        cs.init(cs.OUT, value=1)
        self.spi = spi
        self.dc = dc
        self.res = res
        self.cs = cs
        import time
        self.res(1)
        time.sleep_ms(1)
        self.res(0)
        time.sleep_ms(10)
        self.res(1)
        super().__init__(width, height, external_vcc)

    def write_cmd(self, cmd):
        self.spi.init(baudrate=self.rate, polarity=0, phase=0)
        self.cs(1)
        self.dc(0)
        self.cs(0)
        self.spi.write(bytearray([cmd]))
        self.cs(1)

    def write_data(self, buf):
        self.spi.init(baudrate=self.rate, polarity=0, phase=0)
        self.cs(1)
        self.dc(1)
        self.cs(0)
        self.spi.write(buf)
        self.cs(1)

Sent 147 lines (4584 bytes) to ssd1306.py.


In [11]:
import os
print(os.listdir())

['boot.py', 'hmac.py', 'hotpie.py', 'ssd1306.py']


In [18]:
#https://forum.micropython.org/viewtopic.php?f=18&p=23080
from machine import I2C, Pin
import ssd1306

rst = Pin(16, Pin.OUT)
rst.value(1)
scl = Pin(15, Pin.OUT, Pin.PULL_UP)
sda = Pin(4, Pin.OUT, Pin.PULL_UP)
i2c = I2C(scl=scl, sda=sda, freq=450000)

o = ssd1306.SSD1306_I2C(128, 64, i2c, addr=0x3c)

o.fill(0)
o.text('ESP32', 45, 5)
o.text('Micro Alex Python', 20, 20)
o.show() 

In [80]:
print(i2c.scan())

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'i2c' is not defined


In [81]:
import framebuf

o.fill(0)
for i in range(128):
    o.framebuf.fill_rect(1, 3, i, 34, 1)
    o.show()

Traceback (most recent call last):
  File "<stdin>", line 3, in <module>
NameError: name 'o' is not defined


In [28]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.3-454-g34e224a4 on 2018-03-14; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7fc884a66898, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [84]:
# Show the DoES logo. Scrolls across the screen

import ssd1306
import framebuf
from machine import I2C, Pin
rst = Pin(16, Pin.OUT)
rst.value(1)
scl = Pin(15, Pin.OUT, Pin.PULL_UP)
sda = Pin(4, Pin.OUT, Pin.PULL_UP)
i2c = I2C(scl=scl, sda=sda, freq=450000)
display = ssd1306.SSD1306_I2C(128, 64, i2c, addr=0x3c)
#o = ssd1306.SSD1306_I2C(128, 64, i2c, addr=0x3c)

buffer = bytearray(
    b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xe7\xe0\x30\x0f\xff\xf0\x07\x00\xe0\x30\x03\xff\xf0\x06\x18\xe0\x31\xe3\xe1\xf3\xfe\x3c\xe0\x31\xf1\xc0\x73\xfe\x3f\xe0\x31\xf1\x8c\x73\xff\x07\xe0\x31\xf1\x9e\x30\x07\x80\xe0\x31\xf1\x1e\x30\x0f\xf0\x60\x31\xf1\x1e\x33\xff\xfc\x60\x31\xf3\x9e\x33\xfe\x7e\x60\x31\xc3\x8e\x73\xfe\x3c\x60\x30\x07\xc0\x70\x07\x00\xe0\x30\x1f\xe1\xf0\x07\x81\xe0\x3f\xff\xff\xff\xff\xff\xe0\x3f\xff\xff\xff\xff\xff\xe0\x00\x00\x00\x00\x00\x00\x00\x00\x40\x00\x00\x00\x00\x60\x30\x00\x00\x00\x00\x00\x60\x30\x48\x8c\x15\x86\x0c\x60\x30\x49\xb6\xe6\xcd\x9b\x60\x30\x49\xb2\xc4\x58\xb3\x60\x30\x4d\x3e\xc4\x58\xb3\x60\x30\x47\x30\xc4\x49\xb3\x60\x3f\x46\x1e\xc7\xcf\x1e\x60\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
)
# FRAMEBUF_MHLSB = 3
fb = framebuf.FrameBuffer(buffer, 56, 64,3)

while True:
    for x in range(0,128-56):
        display.fill(0)
        display.blit(fb, x, 0)
        display.show()
    for x in range(0,128-56):
        display.fill(0)
        display.blit(fb, (128-56-x), 0)
        display.show()


.......

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 28, in <module>
KeyboardInterrupt: 
